In [1]:
from __future__ import division, print_function
import sys
import os
%pylab notebook
lib_path = '/home/fgeigl/navigability_of_networks'
sys.path.append(lib_path)
lib_path = '/home/fgeigl/navigability_of_networks/tools'
sys.path.append(lib_path)
import network_matrix_tools
import numpy as np
from scipy.sparse import csr_matrix, diags
import pandas as pd
import datetime
from scipy.sparse.csgraph import connected_components
from collections import Counter
import operator
from sklearn.preprocessing import normalize
import numba
from joblib import Parallel, delayed
from math import sqrt

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']), shape=loader['shape'])

In [3]:
A = load_sparse_csr('/opt/datasets/wiki_clickstream/adjacency_clickstream_network_largest_component.npz')
B = load_sparse_csr('/opt/datasets/wiki_clickstream/clickstream_network_transition_bias_largest_component.npz')    

In [4]:
print('A:', type(A), A.shape)
print('B:', type(B), B.shape)

A: <class 'scipy.sparse.csr.csr_matrix'> (2140423, 2140423)
B: <class 'scipy.sparse.csr.csr_matrix'> (2140423, 2140423)


In [5]:
_, labels = connected_components(A + B, directed=True, connection='strong', return_labels=True)
label_counts = Counter(labels)
largest_label, num_nodes = max(label_counts.items(), key=operator.itemgetter(1))
print('largest component contains', num_nodes, 'nodes', '(', num_nodes/A.shape[0], ')')
if num_nodes != A.shape[0]:
    label_filt = labels == largest_label
    A = A[label_filt, :][:, label_filt]
    B = B[label_filt, :][:, label_filt]
print('A:', type(A), A.shape, A.nnz/(np.power(A.shape[0],2)))
print('B:', type(B), B.shape, B.nnz/(np.power(B.shape[0],2)))

largest component contains 2140423 nodes ( 1.0 )
A: <class 'scipy.sparse.csr.csr_matrix'> (2140423, 2140423) 3.59513059841e-05
B: <class 'scipy.sparse.csr.csr_matrix'> (2140423, 2140423) 2.66174133129e-06


In [6]:
def chunks(my_range, num_chunks):
    chunk_len = int(len(my_range) / num_chunks)
    if len(my_range) % num_chunks > 0:
        chunk_len += 1
    return [my_range[i*chunk_len:(i+1)*chunk_len if i < num_chunks -1 else None] for i in range(num_chunks)]


@numba.jit
def part_dot(M, pi):
    return M.dot(pi)

@numba.jit
def normalize(pi_vec):
    norm = 1./(sqrt(np.power(pi_vec, 2).sum()))
    pi_vec *= norm
    return pi_vec, norm

class ParallelDot:
    def __init__(self, M_chunks):
        self.M_chunks = M_chunks
        self.worker = Parallel(n_jobs=len(M_chunks), batch_size=1, backend='threading')
    
    def dot(self, pi):
        return np.hstack(self.worker(delayed(part_dot)(M_chunk, pi) for M_chunk in self.M_chunks))

def stat_dist_power_iter(M, max_iter = 1e5, precision=10, init_vec = None, n_jobs=5):
    dtype = np.float32 if precision < 7 else (np.float64 if precision < 16 else np.longdouble)
    print('\tusing dtye:', dtype)
    print('\tnormalize...', end='')
    sys.stdout.flush()
    max_iter = int(max_iter)
    M = M.astype(dtype)
    P = M.dot(diags(1. / np.array(M.sum(axis=0), dtype=dtype).flatten()))
    if init_vec is None:
        pi_vec = np.ones(P.shape[0], dtype=dtype) / P.shape[0]
    else:
        pi_vec = init_vec.astype(dtype)
    print('done. ') #, type(pi_vec), pi_vec.dtype, type(P), P.dtype)
    sys.stdout.flush()
    diff = list()
    precision = int(round(precision))
    atol = np.power(1e1, -precision)
    print('\tgen chunk idx...', end='')
    sys.stdout.flush()
    chunk_idx = chunks(range(P.shape[0]), n_jobs)
    assert chunk_idx[-1][-1] == P.shape[0] - 1
    assert chunk_idx[0][0] == 0
    assert sum(map(len, chunk_idx)) == P.shape[0]
    print('done.') # chunk len:', map(len, chunk_idx))    
    sys.stdout.flush()    
    print('\tslice matrix.', P.shape[0], end='...')
    sys.stdout.flush()
    P_row_chunks = [P[idx_range,:] for idx_range in chunk_idx]
    par_dot = ParallelDot(P_row_chunks)
    print('done')
    print('\tstart power iterations. max. iterations:', max_iter)
    sys.stdout.flush()
    comp_times = list()
    start = datetime.datetime.now()
    diverge = 0
    for i in range(1, max_iter + 1):
        now = datetime.datetime.now()
        comp_times.append((now-start).total_seconds())
        comp_times = comp_times[-25:]
        avg_iter_time = sum(comp_times)/len(comp_times)
        start = now
        pi_vec, last_vec = par_dot.dot(pi_vec), pi_vec
        # print(pi_vec)
        pi_vec, norm = normalize(pi_vec)
        current_diff = np.absolute(last_vec - pi_vec).max()
        diff.append(current_diff)
        if len(diff) > 10:
            mean_diff_data = diff[-11:-1]
            mean_diff = sum(mean_diff_data) / len(mean_diff_data)
            improvement = mean_diff - current_diff
            if improvement < 0:
                diverge += 1
                if diverge > 10:
                    print('\n!!!diverge...stopping...')
                    break
            elif mean_diff < atol or improvement * 100. < atol:
                    break
            else:
                diverge = 0
            #time_remain = (current_diff / improvement) * avg_iter_time
            print('\r\titeration:', ("%.0f" % i).rjust(len(str(max_iter)), '0') ,("|| abs iter diff: %." + str(precision) + 'f') % mean_diff,
                  ("|| %.3f" % avg_iter_time), 'sec/it', end='')
                  # '|| min. remain:', datetime.timedelta(seconds=int(time_remain)), end='')
        else:
            print('\r\titeration:', i, end='')
        sys.stdout.flush()
    if i == max_iter:
        print('\n\tdid not converge within', i, 'iterations!!!')
    else:
        print('\n\tneeded', i, 'iterations')
    print('\tlast diff:', (" %." + str(precision) + 'f') % current_diff)
    print('\tlargest eigval:', (" %." + str(precision) + 'f') % norm)
    print('\tsmallest value:', pi_vec.min())
    assert len(pi_vec) == P.shape[0]
    return pi_vec

In [7]:
df_fname = 'click_stream_results_poweriter_10k_p10.df'
time_df_fname = 'click_stream_times_poweriter_10k_p10.df'
try:
    df = pd.read_pickle(df_fname)
    times = pd.read_pickle(time_df_fname)
    print('loaded stored data:', df.columns)
except:
    print('init new data')
    df = pd.DataFrame()
    times = pd.Series()

# stat_dist_power_iter(A)
A = A.astype(np.longdouble)
B = B.astype(np.longdouble)

if 'A_sd' not in df.columns:
    start = datetime.datetime.now()
    #_, df['A_sd'] = network_matrix_tools.calc_entropy_and_stat_dist(A, method='EV', smooth_bias=False, calc_entropy_rate=False)
    df['A_sd'] = stat_dist_power_iter(A)
    times.loc['A_sd'] = datetime.datetime.now() - start
    print(datetime.datetime.now() - start)
init_vec = df['A_sd'].values

loaded stored data: Index([u'beta_0.001', u'beta_0.005', u'beta_0.01', u'beta_0.05', u'beta_0.1',
       u'beta_0.25', u'beta_0.5', u'beta_0.75', u'beta_1.0', u'A_sd'],
      dtype='object')


In [8]:
for beta in [1., 0.75, 0.5, 0.25, 0.1, 0.05, 0.01, 0.005, 0.001]:
    col_name = 'beta_' + str(beta)
    if col_name not in df.columns:
        print('calc beta:', beta)
        print('\t', datetime.datetime.now())
        start = datetime.datetime.now()
        df[col_name] = stat_dist_power_iter((beta * A) + B.T, init_vec = init_vec)
        times.loc[col_name] = datetime.datetime.now() - start
        df.to_pickle(df_fname)
        times.to_pickle(time_df_fname)
    else:
        print('calc beta:', beta, 'already cached')
    print('\ttook:', times.loc[col_name])
    init_vec = df[col_name].values

calc beta: 1.0 already cached
	took: 0 days 00:34:45.895689
calc beta: 0.75 already cached
	took: 0 days 00:39:09.559378
calc beta: 0.5 already cached
	took: 0 days 00:55:31.146890
calc beta: 0.25 already cached
	took: 0 days 01:33:32.803341
calc beta: 0.1 already cached
	took: 0 days 03:03:32.114711
calc beta: 0.05 already cached
	took: 0 days 05:21:16.522985
calc beta: 0.01 already cached
	took: 0 days 17:36:10.518916
calc beta: 0.005 already cached
	took: 0 days 18:55:16.007131
calc beta: 0.001 already cached
	took: 0 days 03:04:57.346407


In [10]:
print('pearson:')
print(df.corr(method='pearson').loc['A_sd'])
print('spearman:')
print(df.corr(method='spearman').loc['A_sd'])
#print('kendall:')
#print(df.corr(method='kendall').iloc[0])

pearson:
beta_0.001    0.031279
beta_0.005    0.090264
beta_0.01     0.176960
beta_0.05     0.546975
beta_0.1      0.649509
beta_0.25     0.724052
beta_0.5      0.772369
beta_0.75     0.801053
beta_1.0      0.821379
A_sd          1.000000
Name: A_sd, dtype: float64
spearman:
beta_0.001    0.536537
beta_0.005    0.542527
beta_0.01     0.548452
beta_0.05     0.580206
beta_0.1      0.606661
beta_0.25     0.657440
beta_0.5      0.706799
beta_0.75     0.738492
beta_1.0      0.761535
A_sd          1.000000
Name: A_sd, dtype: float64


In [ ]:
def sort_key(name):
    val = name.rsplit('_', 1)[-1]
    try:
        return float(val)
    except:
        return 100.

sorted_cols = sorted(df.columns, key=sort_key)
df = df[sorted_cols]
df.to_pickle(df_fname)
times.to_pickle(time_df_fname)
exit()